# Initiate DataFrame and utilities

In [1]:
import seaborn as sns
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from pptx import Presentation
import os

df = pd.read_csv('../data.csv')

def calc_age(row):
    test_date = datetime.strptime(row.loc["test_date"].split('T')[0], "%Y-%m-%d")
    birthdate = datetime.strptime(row.loc["birthdate"].split('T')[0], "%Y-%m-%d")
    age = test_date - birthdate
    return age.days
    
df["age"] = df.apply(calc_age, axis=1)

df = df.loc[(df.age // 365.25 <= 8) & (df.age // 365.25 >= 5)]

In [2]:
# (subtest_lower, subtest, start, end)
subtests = [
    ('dots-comparison', 'Dots Comparison', 1, 10),
    ('match-sample', 'Match Sample', 2, 10),
    ('match-sample-rotate', 'Match Sample Rotate', 2, 10),
    ('match-points-number', 'Match Points Number', 1, 10),
    ('symbolic-magnitude', 'Symbolic Magnitude', 1, 10),
    ('numeric-line', 'Numeric Line', 1, 10),
    ('counting', 'Counting Forward', 1, 9),
    ('counting-true', 'Counting Backward', 1, 19),
    ('simple-arithmetic-plus', 'Simple Arithmetic Plus', 1, 10),
    ('simple-arithmetic-minus', 'Simple Arithmetic Minus', 1, 20),
]

In [3]:
subtests_list = ['dots-comparison','match-sample','match-sample-rotate','match-points-number','symbolic-magnitude','numeric-line','counting','counting-true','simple-arithmetic-plus','simple-arithmetic-minus']

subtest_title = {}
for subtest in subtests_list:
    subtest_title[subtest] = subtest.replace('-', ' ').title()

subtest_title["counting"] = "Counting Forward"
subtest_title["counting-true"] = "Counting Backward"

# Metrics

In [4]:
# we first calculate the average RT when the patient's answer is correct
def calc_rt_correct(row, subtest_lower, start=1, end=10):
    RTs_correct = []
    for i in range(start, end+1):
        if(row[f"{subtest_lower}_{i}_user_score"] == 1 and np.isnan(row[f"{subtest_lower}_{i}_time"]) == False and np.isnan(row[f"{subtest_lower}_{i}_user_score"]) == False):
            RTs_correct.append(row[f"{subtest_lower}_{i}_time"])
    return np.mean(RTs_correct)

In [5]:
for subtest in subtests:
    df[f"{subtest[0]}_RT_correct"] = df.apply(calc_rt_correct, axis=1, subtest_lower=subtest[0], start=subtest[2], end=subtest[3])

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [6]:
# calculate IES correct (in the initial IES, RT instead of RT_correct is used)
# Vandierendonck, 2018

for subtest in subtests:
    df[f"{subtest[0]}_IES_correct"] = df[f"{subtest[0]}_RT_correct"] / df[f"{subtest[0]}_accuracy"]

In [7]:
# calculate RCS
# Vandierendonck, 2018

for subtest in subtests:
    df[f"{subtest[0]}_RCS"] = df[f"{subtest[0]}_accuracy"] / (df[f"{subtest[0]}_RT"] / 1000)

In [8]:
# general-purpouse function to calculate standard deviations

def calc_sd(row, metric, subtest_lower, start=1, end=10, only_correct=False):
    individuals = []
    for i in range(start,end+1):
        if(np.isnan(row[f"{subtest_lower}_{i}_time"]) == False and np.isnan(row[f"{subtest_lower}_{i}_user_score"]) == False):
            if(only_correct):
                if(row[f"{subtest_lower}_{i}_user_score"] == 1):
                    # metric = time
                    individuals.append(row[f"{subtest_lower}_{i}_{metric}"])
            else:
                individuals.append(row[f"{subtest_lower}_{i}_{metric}"])
    return np.std(individuals)

In [9]:
for subtest in subtests:
    df[f"{subtest[0]}_RT_SD"] = df.apply(calc_sd, axis=1, metric="time", subtest_lower=subtest[0], start=subtest[2], end=subtest[3])
    df[f"{subtest[0]}_SCORE_SD"] = df.apply(calc_sd, axis=1, metric="user_score", subtest_lower=subtest[0], start=subtest[2], end=subtest[3])
    df[f"{subtest[0]}_RT_correct_SD"] = df.apply(calc_sd, axis=1, metric="time", subtest_lower=subtest[0], start=subtest[2], end=subtest[3], only_correct=True)

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
# calculate LISAS scores

for subtest in subtests:
    # LISAS for overall metrics
    df[f"{subtest[0]}_LISAS"] = df[f"{subtest[0]}_RT"] + df[f"{subtest[0]}_accuracy"] * (df[f"{subtest[0]}_RT_SD"] / df[f"{subtest[0]}_SCORE_SD"])
    df.loc[df[f"{subtest[0]}_SCORE_SD"] == 0, f"{subtest[0]}_LISAS"] = df[f"{subtest[0]}_RT"]
    
    # LISAS for correct answers
    df[f"{subtest[0]}_LISAS_correct"] = df[f"{subtest[0]}_RT_correct"] + df[f"{subtest[0]}_accuracy"] * (df[f"{subtest[0]}_RT_correct_SD"] / df[f"{subtest[0]}_SCORE_SD"])
    df.loc[df[f"{subtest[0]}_SCORE_SD"] == 0, f"{subtest[0]}_LISAS_correct"] = df[f"{subtest[0]}_RT_correct"]

# Tables

## General

In [43]:
writer = pd.ExcelWriter("tables/metrics.xlsx", engine = 'xlsxwriter')

for subtest in subtests:
    columns = ["5 y/o", "6 y/o", "7 y/o", "8 y/o"]
    rows = ["LISAS", "LISAS when accurate", "IES", "IES when accurate", "RCS", "Accuracy", "RT"]
    df_subtest = pd.DataFrame(columns = columns, index = rows)
    for age in range(5,9):
        df_subtest.loc["LISAS", f"{age} y/o"] = df.loc[df.age // 365.25 == age, f"{subtest[0]}_LISAS"].mean()
        df_subtest.loc["LISAS when accurate", f"{age} y/o"] = df.loc[df.age // 365.25 == age, f"{subtest[0]}_LISAS_correct"].mean()
        df_subtest.loc["IES", f"{age} y/o"] = df.loc[df.age // 365.25 == age, f"{subtest[0]}_IES"].mean()
        df_subtest.loc["IES when accurate", f"{age} y/o"] = df.loc[df.age // 365.25 == age, f"{subtest[0]}_IES_correct"].mean()
        df_subtest.loc["RCS", f"{age} y/o"] = df.loc[df.age // 365.25 == age, f"{subtest[0]}_RCS"].mean()
        df_subtest.loc["Accuracy", f"{age} y/o"] = df.loc[df.age // 365.25 == age, f"{subtest[0]}_accuracy"].mean()
        df_subtest.loc["RT", f"{age} y/o"] = df.loc[df.age // 365.25 == age, f"{subtest[0]}_RT"].mean()
    df_subtest.to_excel(writer, sheet_name = subtest[1].replace(" ", "-").lower())

writer.save()
writer.close()

## Correlation matrices

In [35]:
df_LISAS = df.loc[:,(col for col in df.columns if "LISAS" in col and "correct" not in col)]
df_LISAS.corr().to_excel("tables/LISAS_correlation.xlsx")

df_LISAS_correct = df.loc[:,(col for col in df.columns if "LISAS" in col and "correct" in col)]
df_LISAS_correct.corr().to_excel("tables/LISAS_correct_correlation.xlsx")

df_IES = df.loc[:,(col for col in df.columns if "IES" in col and "correct" not in col)]
df_IES.corr().to_excel("tables/IES_correlation.xlsx")

df_IES_correct = df.loc[:,(col for col in df.columns if "IES" in col and "correct" in col)]
df_IES_correct.corr().to_excel("tables/IES_correct_correlation.xlsx")

df_RCS = df.loc[:,(col for col in df.columns if "RCS" in col)]
df_RCS.corr().to_excel("tables/RCS_correlation.xlsx")

# Charts

_"Individual"_ means each subtest individually.

_"General"_ means all subtests together.

## Individual

In [36]:
# normal plots

for subtest in subtests:
    dir_charts = "charts/" + subtest[1].replace(" ", "-").lower() + "/"
    if not os.path.exists(dir_charts):
        os.makedirs(dir_charts)

    plt.title(f"{subtest[1]} LISAS by age")
    plt.xlabel("Age")
    plt.ylabel("Average LISAS")

    ages = [5,6,7,8]
    x_labels = ["5 y/o", "6 y/o", "7 y/o", "8 y/o"]

    # graph LISAS
    y_points_LISAS_overall = [df.loc[df.age // 365.25 == age, f"{subtest[0]}_LISAS"].mean() for age in ages]
    plt.plot(x_labels, y_points_LISAS_overall, label="General")
    y_points_LISAS_correct = [df.loc[df.age // 365.25 == age, f"{subtest[0]}_LISAS_correct"].mean() for age in ages]
    plt.plot(x_labels, y_points_LISAS_correct, label="When accurate")

    plt.legend()
    plt.savefig(dir_charts + f"{subtest[1]} LISAS by age".replace(" ", "-").lower() + ".png")
    plt.clf()

    # graph IES
    plt.title(f"{subtest[1]} IES by age")
    plt.xlabel("Age")
    plt.ylabel("Average IES")

    y_points_IES_overall = [df.loc[df.age // 365.25 == age, f"{subtest[0]}_IES"].mean() for age in ages]
    plt.plot(x_labels, y_points_IES_overall, label="General")
    y_points_IES_correct = [df.loc[df.age // 365.25 == age, f"{subtest[0]}_IES_correct"].mean() for age in ages]
    plt.plot(x_labels, y_points_IES_correct, label="When accurate")
    
    plt.legend()
    plt.savefig(dir_charts + f"{subtest[1]} IES by age".replace(" ", "-").lower() + ".png")
    plt.clf()

    # graph RCS
    plt.title(f"{subtest[1]} RCS by age")
    plt.xlabel("Age")
    plt.ylabel("Average RCS")

    y_points_IES = [df.loc[df.age // 365.25 == age, f"{subtest[0]}_RCS"].mean() for age in ages]
    plt.plot(x_labels, y_points_IES)
    plt.savefig(dir_charts + f"{subtest[1]} RCS by age".replace(" ", "-").lower() + ".png")
    plt.clf()

<Figure size 432x288 with 0 Axes>

In [79]:
# box plots

for subtest in subtests:
    dir_charts = "charts/" + subtest[1].replace(" ", "-").lower() + "/"
    if not os.path.exists(dir_charts):
        os.makedirs(dir_charts)

    plt.xlabel("Age (years)")

    ages = [5,6,7,8]
    x_labels = ["5 y/o", "6 y/o", "7 y/o", "8 y/o"]

    for metric in ["LISAS", "LISAS correct", "IES", "IES correct", "RCS"]:
        plt.title(f"{subtest[1]} {metric} by age".replace("correct", "when accurate"))
        plt.ylabel(f"Average {metric}".replace("correct", "when accurate"))
        scores_by_age = []
        for age in ages:
            scores_by_age.append(df.loc[df.age // 365.25 == age, f"{subtest[0]}_{metric.replace(' ', '_')}"])
        scores_by_age_df = pd.DataFrame({"5": scores_by_age[0], "6": scores_by_age[1], "7": scores_by_age[2], "8": scores_by_age[3]})
        boxplot = scores_by_age_df.boxplot(showmeans = True, meanprops={"marker":"o", "markerfacecolor":"blue", "markeredgecolor":"blue", "markersize":"3"})
        plt.savefig("charts/" + subtest[1].replace(" ", "-").lower() + f"/{subtest[1].replace(' ', '-').lower()}_" + metric.replace(" ", "_") + "_boxplot.png", bbox_inches='tight')
        plt.clf()

<Figure size 432x288 with 0 Axes>

## General

In [37]:
# standard plots
flierprops = dict(marker='o', markersize=2, linestyle='none')

for metric in ["LISAS", "LISAS correct", "IES", "IES correct", "RCS"]:
    df_metric = pd.DataFrame()
    if "correct" in metric:
        df_metric = df.loc[:,(col for col in df.columns if metric.split(" ")[0] in col and "correct" in col)]
    else:
        df_metric = df.loc[:,(col for col in df.columns if metric in col and "correct" not in col)]
    df_metric=(df_metric-df_metric.mean())/df_metric.std()
    df_metric.columns = [subtest_title[col.split("_")[0]] for col in df_metric.columns]
    boxplot = df_metric.boxplot(flierprops=flierprops, vert=False)
    # plt.title(metric)
    plt.savefig("charts/" + metric.replace(" ", "_") + "_boxplot.png", bbox_inches='tight')
    plt.clf()

<Figure size 432x288 with 0 Axes>

# Presentation

In [55]:
def add_slide(picture_path, title, subtitle, slide_index=0):
    slide = prs.slides.add_slide(prs.slide_layouts[slide_index])
    subtest_title = slide.placeholders[0]
    chart = slide.placeholders[13]
    metric_measured = slide.placeholders[14]
    
    chart.insert_picture(picture_path)
    subtest_title.text = title
    metric_measured.text = subtitle

In [62]:
# original presentation

prs = Presentation('presentation.pptx')

for subtest in subtests:
    dir_charts = "charts/" + subtest[1].replace(" ", "-").lower() + "/"
    add_slide(dir_charts + f"{subtest[1]} LISAS by age".replace(" ", "-").lower() + ".png", subtest[1], "LISAS by age")
    add_slide(dir_charts + f"{subtest[1]} IES by age".replace(" ", "-").lower() + ".png", subtest[1], "IES by age")
    add_slide(dir_charts + f"{subtest[1]} RCS by age".replace(" ", "-").lower() + ".png", subtest[1], "RCS by age")
    add_slide("../jan18,2022/charts/" + subtest[1].replace(" ", "-").lower() + "/" + subtest[1].replace(" ", "-").lower() + "_general_rt.png", subtest[1], "RT by age")
    add_slide("../jan18,2022/charts/" + subtest[1].replace(" ", "-").lower() + "/" + subtest[1].replace(" ", "-").lower() + "_general_accuracy.png", subtest[1], "Accuracy by age")

for metric in ["LISAS", "LISAS correct", "IES", "IES correct", "RCS"]:
    add_slide("charts/" + metric.replace(" ", "_") + "_boxplot.png", metric, "Box plot by subtest", slide_index=1)

prs.save('presentation.pptx')

In [80]:
# presentation update using boxplots

prs = Presentation('presentation_update.pptx')

for subtest in subtests:
    dir_charts = "charts/" + subtest[1].replace(" ", "-").lower() + "/"
    add_slide(dir_charts + subtest[1].replace(" ", "-").lower() + "_LISAS_boxplot.png", subtest[1], "LISAS by age")
    add_slide(dir_charts + subtest[1].replace(" ", "-").lower() + "_LISAS_correct_boxplot.png", subtest[1], "LISAS when accurate by age")
    add_slide(dir_charts + subtest[1].replace(" ", "-").lower() + "_IES_boxplot.png", subtest[1], "IES by age")
    add_slide(dir_charts + subtest[1].replace(" ", "-").lower() + "_IES_correct_boxplot.png", subtest[1], "IES when accurate by age")
    add_slide(dir_charts + subtest[1].replace(" ", "-").lower() + "_RCS_boxplot.png", subtest[1], "RCS by age")

prs.save('presentation_update.pptx')

# Export separate data

In [40]:
df_LISAS = df.loc[:,(col for col in df.columns if "LISAS" in col and "correct" not in col)]
df_LISAS.to_excel("tables/dataframes/LISAS.xlsx")
df_LISAS.to_csv("tables/dataframes/LISAS.csv")

df_LISAS_correct = df.loc[:,(col for col in df.columns if "LISAS" in col and "correct" in col)]
df_LISAS_correct.to_excel("tables/dataframes/LISAS_correct.xlsx")
df_LISAS_correct.to_csv("tables/dataframes/LISAS_correct.csv")

df_IES = df.loc[:,(col for col in df.columns if "IES" in col and "correct" not in col)]
df_IES.to_excel("tables/dataframes/IES.xlsx")
df_IES.to_csv("tables/dataframes/IES.csv")

df_IES_correct = df.loc[:,(col for col in df.columns if "IES" in col and "correct" in col)]
df_IES_correct.to_excel("tables/dataframes/IES_correct.xlsx")
df_IES_correct.to_csv("tables/dataframes/IES_correct.csv")

df_RCS = df.loc[:,(col for col in df.columns if "RCS" in col)]
df_RCS.to_excel("tables/dataframes/RCS.xlsx")
df_RCS.to_csv("tables/dataframes/RCS.csv")


# Extra

## Compare outliers

In [17]:
from scipy import stats

writer_ids = pd.ExcelWriter("tables/outliers_uncolored.xlsx", engine = 'xlsxwriter')
writer_counts = pd.ExcelWriter("tables/outliers_counts.xlsx", engine = 'xlsxwriter')

for metric in ["LISAS", "LISAS correct", "IES", "IES correct", "RCS"]:
    df_outliers = pd.DataFrame()
    for idx, subtest in enumerate(subtests):
        z = pd.Series(stats.zscore(df[f"{subtest[0]}_{metric.replace(' ', '_')}"], nan_policy="omit"))
        Q1 = z.quantile(0.25)
        Q3 = z.quantile(0.75)
        IQR = Q3 - Q1
        outliers = z[z > Q3 + 1.5 * IQR]
        series_outliers = pd.Series(outliers.index, name=subtest[1])
        df_outliers = pd.concat([df_outliers, series_outliers], axis=1)
    df_outliers.to_excel(writer_ids, sheet_name = metric, index=False)

    counted_values = df_outliers[df_outliers.columns[0]].value_counts()
    for column in df_outliers.columns[1:]:
        counted_values = counted_values.add(df_outliers[column].value_counts(), fill_value=0)
    counted_values.sort_values(ascending=False).to_excel(writer_counts, sheet_name = metric)

writer_ids.save()
writer_ids.close()
writer_counts.save()
writer_counts.close()

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [78]:
counted_values = df_outliers[df_outliers.columns[0]].value_counts()

for column in df_outliers.columns[1:]:
    counted_values = counted_values.add(df_outliers[column].value_counts(), fill_value=0)

counted_values.sort_values(ascending=False).to_excel("documents/outliers_counted.xlsx")